![CH4-ADS.png](.\Media\CH4-ADS.png)

## Get information on location, time and size of any memory dumps from SQL Server (Memory Dump Info)

### <span style="color:rgb(0, 204, 153);">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Get information on location, time and size of any memory dumps from SQL Server (Memory Dump Info)
SELECT [filename], creation_time, size_in_bytes/1048576.0 AS [Size (MB)]
FROM sys.dm_server_memory_dumps WITH (NOLOCK) 
ORDER BY creation_time DESC OPTION (RECOMPILE);

This will not return any rows if you have not had any memory dumps (which is a good thing)

[sys.dm\_server\_memory\_dumps (Transact-SQL)](https://bit.ly/2elwWll)

## Page Life Expectancy (PLE) value for each NUMA node in current instance (PLE by NUMA Node)

### <span style="color:rgb(0, 204, 153);">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Page Life Expectancy (PLE) value for each NUMA node in current instance (PLE by NUMA Node)
SELECT @@SERVERNAME AS [Server Name], RTRIM([object_name]) AS [Object Name], 
       instance_name, cntr_value AS [Page Life Expectancy]
FROM sys.dm_os_performance_counters WITH (NOLOCK)
WHERE [object_name] LIKE N'%Buffer Node%' -- Handles named instances
AND counter_name = N'Page life expectancy' OPTION (RECOMPILE);

<span style="color: rgb(255, 255, 254); white-space: normal;">PLE is a good measurement of internal memory pressure</span>

- Higher PLE is better. Watch the trend over time, not the absolute value
- This will only return one row for non-NUMA systems

[Page Life Expectancy isn’t what you think…](https://bit.ly/2EgynLa)

## Memory Grants Pending value for current instance (Memory Grants Pending)

### <span style="color:rgb(0, 204, 153);">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Memory Grants Pending value for current instance (Memory Grants Pending)
SELECT @@SERVERNAME AS [Server Name], RTRIM([object_name]) AS [Object Name], cntr_value AS [Memory Grants Pending]                                                                                                       
FROM sys.dm_os_performance_counters WITH (NOLOCK)
WHERE [object_name] LIKE N'%Memory Manager%' -- Handles named instances
AND counter_name = N'Memory Grants Pending' OPTION (RECOMPILE);

Run multiple times, and run periodically if you suspect you are under memory pressure

- Memory Grants Pending above zero for a sustained period is a very strong indicator of internal memory pressure

## Memory Clerk Usage for instance (Memory Clerk Usage)

### <span style="color:rgb(0, 204, 153);">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Memory Clerk Usage for instance (Memory Clerk Usage)
-- Look for high value for CACHESTORE_SQLCP (Ad-hoc query plans)
SELECT TOP(10) mc.[type] AS [Memory Clerk Type], 
       CAST((SUM(mc.pages_kb)/1024.0) AS DECIMAL (15,2)) AS [Memory Usage (MB)] 
FROM sys.dm_os_memory_clerks AS mc WITH (NOLOCK)
GROUP BY mc.[type]  
ORDER BY SUM(mc.pages_kb) DESC OPTION (RECOMPILE);

**MEMORYCLERK\_SQLBUFFERPOOL** was new for SQL Server 2012. It should be your highest consumer of memory

**CACHESTORE\_SQLCP** - SQL Plans

- These are cached SQL statements or batches that aren't in stored procedures, functions and triggers
- Watch out for high values for CACHESTORE\_SQLCP
- Enabling 'optimize for ad hoc workloads' at the instance level can help reduce this
- Running DBCC FREESYSTEMCACHE ('SQL Plans') periodically may be required to better control this

**CACHESTORE\_OBJCP** - Object Plans

- These are compiled plans for stored procedures, functions and triggers

[sys.dm\_os\_memory\_clerks (Transact-SQL)](https://bit.ly/2H31xDR)